# Generador de personas aleatorias

In [201]:
import pandas as pd
from faker import Faker
import numpy as np
import random as rm
import datetime as dt
import unicodedata

In [202]:
reap = pd.read_csv('./conjunto_datos/relacion_edad_altura_peso.csv')
nombres = pd.read_csv('./catalogos/nombres.csv')
profesiones = pd.read_csv('./catalogos/profesiones.csv')

def quitar_acentos(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [258]:
fake = Faker(['es_MX'])
for _ in range(1):
    sexo = rm.choice(['H','M'])
    # generar tipo de sangre y asignar probabilidad a cada valor para que salga
    tipo_sangre = rm.choices(['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-'], weights = [30, 6, 9, 2, 4, 1, 39, 9])
    tipo_sangre = tipo_sangre[0]

    fecha_nac = fake.date_of_birth(minimum_age = int(reap['edad'].min()))
    # calcular edad en base a la fecha de nacimiento
    dia_actual = dt.date.today()
    edad = dia_actual.year - fecha_nac.year - ((dia_actual.month, dia_actual.day) < (fecha_nac.month, fecha_nac.day))

    # dependiendo de la edad se busca el margen de alturas y pesos que tenga esa edad
    # si es mayor que la edad maxima que este registrada se le da el margen de esa edad maxima
    # mucha gente deja de crecer a los 20
    margen_ap = reap[(reap['edad'] == edad) if edad <= reap['edad'].max() else (reap['edad'] == reap['edad'].max())].reset_index()

    # dependiendo del sexo se generan nombres femeninos o masculinos
    # y estaturas y pesos dentro de su margen correspondiente
    if sexo == 'H':
        titulo = rm.choice(['Sr.', 'Dr.', 'Mtro.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else ''
        nombre = rm.choice(nombres['h'])
        altura = round(rm.uniform(margen_ap['lim_inf.altura.h.cm'][0], margen_ap['lim_sup.altura.h.cm'][0]), 2)
        peso = round(rm.uniform(margen_ap['lim_inf.peso.h.kg'][0], margen_ap['lim_sup.peso.h.kg'][0]), 2)
    else:
        titulo = rm.choice(['Sra.', 'Dra.', 'Mtra.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else ''
        nombre = rm.choice(nombres['m'])
        altura = round(rm.uniform(margen_ap['lim_inf.altura.m.cm'][0], margen_ap['lim_sup.altura.m.cm'][0]), 2)
        peso = round(rm.uniform(margen_ap['lim_inf.peso.m.kg'][0], margen_ap['lim_sup.peso.m.kg'][0]), 2)
    
    apellido_paterno = fake.last_name()
    apellido_materno = fake.last_name()
    
    # Generar un nombre de usuario para un email
    rand_int = rm.randint(1, 4)
    if rand_int == 1:
        usuario = f'{nombre[0]}{apellido_paterno}'
    elif rand_int == 2:
        usuario = f'{nombre}_{apellido_paterno}'
    elif rand_int == 3:
        anio_correo = fecha_nac.strftime('%y')
        usuario = f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{anio_correo}'
    elif rand_int == 4:
        usuario = fake.user_name()
    usuario = usuario.replace(' ', rm.choice(['', '_'])).lower()
    usuario = quitar_acentos(usuario)
    subdominio = rm.choice(['.com', '.net', '.io', '.gob', '.org', '.edu'])
    correo = usuario + f'@example{subdominio}'
    
    lada = str(rm.randint(100, 999))
    cel = lada + str(rm.randint(1000000, 9999999))
    profesion = list(profesiones.sample())[0] if edad >= 18 else ''
    
    if titulo: print(titulo, nombre, apellido_paterno, apellido_materno)
    else: print(nombre, apellido_paterno, apellido_materno)
    print('sexo', sexo, 'tipo de sangre', tipo_sangre)
    print('fecha nacimiento', fecha_nac, 'edad', edad)
    print('altura', altura, 'peso', peso)
    print('correo', correo)
    print('cel', cel)
    if profesion: print('profesion', profesion)
    print('')

    direccion = fake.street_address()
    direcc_calle = fake.street_name()
    direcc_numero = fake.building_number()
    direcc_cp = fake.postcode()
    ciudad = fake.city()
    estado = fake.state()
    lat = fake.latitude()
    lon = fake.longitude()
    timezone = fake.timezone()
    color_favorito = fake.safe_color_name()

    print(direccion)
    print('')
    print(direcc_calle, direcc_numero)
    print('CP', direcc_cp)
    print(ciudad, estado)
    print(lat, lon)
    print('zona horaria', timezone)


Ilse Ybarra Farías
sexo M tipo de sangre O+
fecha nacimiento 1927-05-29 edad 94
altura 172.57 peso 69.4
correo iyf27@example.net
cel 1751927412
profesion Bibliotecario académico

Ampliación Godínez 203 857

Avenida Canadá 5301
CP 30289-6092
San Rodrigo los altos Zacatecas
-21.144670 -176.808797
zona horaria America/Tegucigalpa


In [68]:
# mas info en https://randomuser.me/documentation
url_api = 'https://randomuser.me/api?format=csv'
params = {
    'results': 100,
    #'nat': 'es',
    #'password': '8-16,lower,upper,number,special',
    'exc': 'nat,picture,id,login,registered'
}
query = ''
for k, v in params.items():
    if isinstance(v, int): v = str(v)
    query += '&' + k + '=' + v
url = url_api + query

personas = pd.read_csv(url)
personas.sample(8)

,gender,name.title,name.first,name.last,location.street.number,location.street.name,location.city,location.state,location.country,location.postcode,location.coordinates.latitude,location.coordinates.longitude,location.timezone.offset,location.timezone.description,email,dob.date,dob.age,phone,cell
82,female,Ms,Candice,Gordon,766,Miller Ave,Santa Rosa,Maine,United States,88037,-23.3753,-58.6174,+4:30,Kabul,candice.gordon@example.com,1956-03-26T14:13:22.070Z,66,(161)-531-2969,(866)-566-8973
9,female,Mademoiselle,Julie,Petit,7093,Place de L'Abbé-Georges-Hénocque,Welschenrohr,Solothurn,Switzerland,2662,-36.4207,-135.3919,+9:00,"Tokyo, Seoul, Osaka, Sapporo, Yakutsk",julie.petit@example.com,1948-11-13T22:21:57.196Z,74,077 413 75 71,078 964 04 14
24,male,Mr,Michael,Li,3376,Main Street East,Dunedin,Canterbury,New Zealand,40128,-86.4914,103.4236,+10:00,"Eastern Australia, Guam, Vladivostok",michael.li@example.com,1962-07-18T17:53:10.753Z,60,(693)-697-2830,(361)-381-5730
94,male,Mr,Chris,Nguyen,480,First Street,Traralgon,South Australia,Australia,5996,55.4361,57.1844,+11:00,"Magadan, Solomon Islands, New Caledonia",chris.nguyen@example.com,1967-07-05T03:10:03.581Z,55,02-6770-9886,0499-442-822
1,male,Mr,Jose,Diaz,3370,Richmond Road,Bristol,North Yorkshire,United Kingdom,H3 1TL,88.9311,31.7568,+3:30,Tehran,jose.diaz@example.com,1972-07-30T21:52:50.113Z,50,0115691 939 3989,0737-848-824
41,male,Mr,Ceyhun,Kavaklıoğlu,3871,Kushimoto Sk,Samsun,Kütahya,Turkey,44029,-5.7726,-150.1974,+4:30,Kabul,ceyhun.kavaklioglu@example.com,1957-08-07T17:53:53.303Z,65,(953)-582-9600,(450)-475-5410
87,male,Mr,مانی,سلطانی نژاد,9713,آفریقا,تهران,اردبیل,Iran,19278,67.5845,77.6546,-2:00,Mid-Atlantic,mny.sltnynjd@example.com,1982-05-31T18:31:46.558Z,40,075-17146465,0997-792-2518
0,female,Mademoiselle,Nora,Joly,853,Rue Jean-Baldassini,Saicourt,Nidwalden,Switzerland,9063,-51.1860,37.2472,0:00,"Western Europe Time, London, Lisbon, Casablanca",nora.joly@example.com,1994-04-25T12:42:54.968Z,28,078 141 19 55,077 334 45 22
